In [26]:
import duckdb as duck
from openbb import obb
import polars as pl

import os
# os.chdir('..')
print(os.getcwd())
import sys 
sys.path.append('./working_files/')
from functions import *



stocks = [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "GOOGL", # Alphabet Inc. (Class A)
    "AMZN",  # Amazon.com Inc.
    "META",  # Meta Platforms Inc.
    "NVDA",  # NVIDIA Corporation
    "TSLA",  # Tesla Inc.
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "AMD", # AMD 
    "KO", # Coca Cola
    "SBUX", #Starbucks
    "PEP", # Pepsi
    "GE", # General Electric
    "GM", # General Motors
    "NFLX", # Netflix
    "RBLX", # Roblox
    "SONY", # Sony
    "WMT", # Walmart
    "IBM", # IBM
    "TGT", # target
    "COF" ,# Capital One,
    "PLTR", # Palantir,
    "MELI", # MercadoLibre
    "ROAD", #Construction
    "QUBT", # Quantum Computing
    "SHOP", # Shopify
    "ADBE" ,# Adobe
    "BRK-B",# berkshire hathaway
    "OKLO", # Oklo Inc
    "SMR" # Nuscale Power Corp
]

current_working = os.getcwd()
con = duck.connect(f"{current_working}/database/stock_data.db")

# con.execute('drop table AAPL')

for stock in stocks:
    con.execute(f"""
                CREATE TABLE IF NOT EXISTS "{stock}" (
                date DATE PRIMARY KEY,
                open FLOAT,
                high FLOAT, 
                low FLOAT,
                close FLOAT,
                volume INT,
                daily_return FLOAT,
                moving_average float,
                ema FLOAT,
                vwap_proxy FLOAT
                )
                """)


/Users/jasonrodriguez/economic_outlook


ModuleNotFoundError: No module named 'functions'

In [22]:
database = []


con = duck.connect(database=":memory:")

for stock in stocks:
    # Fetch and process data
    stock_data = adding_relevant_columns(
        obb.equity.price.historical(
            symbol=stock, provider='yfinance', start_date='2007-01-01'
        ).to_df()
    )
    stock_data = (
        pl.from_pandas(stock_data.reset_index())
        .with_columns([pl.lit(f"{stock}").alias("symbol")])
    )
    recent_date = max(stock_data['date']).strftime('%Y-%m-%d')
    min_date = min(stock_data['date']).strftime('%Y-%m-%d')

    con.execute(f"""
        CREATE TABLE IF NOT EXISTS "{stock}" (
            date DATE PRIMARY KEY,
            open FLOAT,
            high FLOAT, 
            low FLOAT,
            close FLOAT,
            volume BIGINT,
            daily_return FLOAT,
            moving_average FLOAT,
            ema FLOAT,
            vwap_proxy FLOAT
        )
    """)

    con.register("stock_data_view", stock_data)

    con.execute(f'''
        INSERT INTO "{stock}" (
            date, open, high, low, close, volume, daily_return, moving_average, ema, vwap_proxy
        )
        SELECT date, open, high, low, close, volume, daily_return, moving_average, ema, vwap_proxy
        FROM stock_data_view
        WHERE date <= DATE '{recent_date}'
        and date >= DATE '{min_date}'
    ''')


NameError: name 'stocks' is not defined

In [14]:
con.sql('''
            SELECT * FROM 
            googl
            order by 1 asc
            limit 10
            ''').show()

CatalogException: Catalog Error: Table with name googl does not exist!
Did you mean "pg_class"?

In [3]:
stock_data = adding_relevant_columns(
    obb.equity.price.historical(
        symbol='AAPL', provider='yfinance', start_date='2007-01-01'
    ).to_df()
)
apple = (
    pl.from_pandas(stock_data.reset_index())
    .with_columns([pl.lit(f"{stock}").alias("symbol")])
)

max(apple['date']).strftime('%Y-%m-%d')

NameError: name 'adding_relevant_columns' is not defined